### 핵심 동작 원리

첫 번째 카드: 항상 랜덤 선택
기억 검색: 첫 번째 카드와 같은 색상을 기억에서 찾기
두 번째 카드 선택:

기억에 있으면 → 그 위치로 직접 이동 <br>
기억에 없으면 → 랜덤 선택


메모리 관리: FIFO 방식으로 오래된 것부터 제거
매칭 성공시: 해당 카드들을 기억에서 제거

## 현재 게임 규칙

1단계2x2 / 총 그림 2개- 기본 카드: 2개- 트릭 카드: 0개제한 시간: 5초성공 리워드: 2원실패 리워드: 1원

2단계2x3 / 총 그림 3개- 기본 카드: 3개- 트릭 카드: 0개제한 시간: 7초성공 리워드: 2원

3단계3x4 / 총 그림 6개- 기본 카드: 6개- 트릭 카드: 0개제한 시간: 9초성공 리워드: 2원

4단계4x4 / 총 그림 8개- 기본 카드: 8개- 트릭 카드: 0개제한 시간: 13초성공 리워드: 2원

5단계4x4 / 총 그림 8개- 기본 카드: 6개- 트릭 카드: 2개제한 시간: 15초성공 리워드: 3원

6단계4x5 / 총 그림 10개- 기본 카드: 10개- 트릭 카드: 0개제한 시간: 21초성공 리워드: 3원

7단계4x5 / 총 그림 10개- 기본 카드: 8개- 트릭 카드: 2개제한 시간: 25초성공 리워드: 6원

- 재참여 : 최대 3번까지 플레이 가능. 이 경우, 1단계부터 재시작하며 광고를 시청한다. 매출: 1원
- 재시도 : 무재한 재시도 가능. 이 경우, 기존 실패한 단계에서 재시작하며 광고를 시청 한다. 매출 : 1원

In [109]:
import random
import numpy as np
from collections import deque, Counter
from typing import List, Tuple, Dict
import pandas as pd

class Card:
    """카드 클래스"""
    def __init__(self, color: int, position: int):
        self.color = color
        self.position = position
        self.is_matched = False

class LimitedMemoryPlayer:
    """최근 N개 카드만 기억하는 플레이어 클래스"""
    def __init__(self, memory_size: int = 3):
        self.memory_size = memory_size
        self.memory = {}  # position -> color 매핑
        self.memory_order = deque(maxlen=memory_size)  # FIFO 순서 관리
        
    def remember_card(self, position: int, color: int):
        """카드를 기억에 추가"""
        if position in self.memory:
            self.memory_order.remove(position)
            self.memory_order.append(position)
        else:
            if len(self.memory) >= self.memory_size:
                oldest_pos = self.memory_order.popleft()
                del self.memory[oldest_pos]
            
            self.memory[position] = color
            self.memory_order.append(position)
    
    def find_matching_card(self, target_color: int, exclude_positions: List[int] = None) -> int:
        """기억에서 같은 색상의 카드 위치 찾기"""
        if exclude_positions is None:
            exclude_positions = []
            
        for position, color in self.memory.items():
            if color == target_color and position not in exclude_positions:
                return position
        return -1
    
    def remove_from_memory(self, positions: List[int]):
        """매칭된 카드들을 기억에서 제거"""
        for pos in positions:
            if pos in self.memory:
                del self.memory[pos]
                if pos in self.memory_order:
                    self.memory_order.remove(pos)

def create_deck(n_pairs: int) -> List[Card]:
    """n개 쌍의 카드 덱 생성"""
    deck = []
    colors = list(range(n_pairs)) * 2
    random.shuffle(colors)
    
    for i, color in enumerate(colors):
        deck.append(Card(color, i))
    
    return deck

def get_available_positions(deck: List[Card]) -> List[int]:
    """아직 매칭되지 않은 카드 위치들 반환"""
    return [i for i, card in enumerate(deck) if not card.is_matched]

def simulate_single_game(n_pairs: int, memory_size: int) -> Tuple[int, bool]:
    """단일 게임 시뮬레이션"""
    deck = create_deck(n_pairs)
    player = LimitedMemoryPlayer(memory_size)
    turns = 0
    matched_pairs = 0
    
    while matched_pairs < n_pairs:
        turns += 1
        
        # 사용 가능한 카드들
        available_positions = get_available_positions(deck)
        
        if len(available_positions) < 2:
            break
        
        # 첫 번째 카드 선택 (랜덤)
        first_pos = random.choice(available_positions)
        first_card = deck[first_pos]
        
        # 첫 번째 카드를 기억에 추가
        player.remember_card(first_pos, first_card.color)
        
        # 기억에서 같은 색상의 카드 찾기
        matching_pos = player.find_matching_card(first_card.color, [first_pos])
        
        if matching_pos != -1 and matching_pos in available_positions:
            # 기억에서 매칭 카드를 찾은 경우
            second_pos = matching_pos
            second_card = deck[second_pos]
            
            # 매칭 성공
            first_card.is_matched = True
            second_card.is_matched = True
            matched_pairs += 1
            
            # 매칭된 카드들을 기억에서 제거
            player.remove_from_memory([first_pos, second_pos])
        else:
            # 기억에 없으므로 랜덤하게 두 번째 카드 선택
            available_for_second = [pos for pos in available_positions if pos != first_pos]
            
            if not available_for_second:
                break
                
            second_pos = random.choice(available_for_second)
            second_card = deck[second_pos]
            
            # 두 번째 카드를 기억에 추가
            player.remember_card(second_pos, second_card.color)
            
            if first_card.color == second_card.color:
                # 운 좋게 매칭
                first_card.is_matched = True
                second_card.is_matched = True
                matched_pairs += 1
                
                # 매칭된 카드들을 기억에서 제거
                player.remove_from_memory([first_pos, second_pos])
    
    return turns, matched_pairs == n_pairs

def calculate_turn_probability_density(turns_list: List[int], max_turns: int) -> Dict[int, float]:
    """각 턴당 확률 밀도 계산"""
    turn_counter = Counter(turns_list)
    total_games = len(turns_list)
    
    probability_density = {}
    for turn in range(1, max_turns + 1):
        count = turn_counter.get(turn, 0)
        probability_density[f"turn_{turn}"] = count / total_games
    
    return probability_density

def run_detailed_simulation(n_pairs: int, memory_size: int, num_simulations: int = 10000) -> Dict:
    """상세 결과를 포함한 시뮬레이션 실행"""
    turns_results = []
    successful_games = 0
    
    # 시뮬레이션 실행
    for _ in range(num_simulations):
        turns, success = simulate_single_game(n_pairs, memory_size)
        turns_results.append(turns)
        if success:
            successful_games += 1
    
    # 기본 통계 계산
    min_turns = min(turns_results)
    max_turns = max(turns_results)
    average_turns = np.mean(turns_results)
    std_turns = np.std(turns_results)
    success_rate = successful_games / num_simulations
    
    # 확률 밀도 계산
    probability_density = calculate_turn_probability_density(turns_results, max_turns)
    
    # 결과 딕셔너리 구성
    result = {
        'n_pairs': n_pairs,
        'memory_size': memory_size,
        'num_simulations': num_simulations,
        'min_turns': min_turns,
        'max_turns': max_turns,
        'average_turns': round(average_turns, 3),
        'std_turns': round(std_turns, 3),
        'success_rate': round(success_rate, 4),
        **probability_density  # turn_1, turn_2, ... 확률 밀도 추가
    }
    
    return result

def run_batch_detailed_simulation(n_pairs_list: List[int], 
                                memory_sizes: List[int], 
                                num_simulations: int = 10000) -> List[Dict]:
    """여러 조건에 대한 배치 시뮬레이션"""
    results = []
    
    for n_pairs in n_pairs_list:
        for memory_size in memory_sizes:
            print(f"시뮬레이션 중: {n_pairs}쌍, 메모리 크기 {memory_size}...")
            result = run_detailed_simulation(n_pairs, memory_size, num_simulations)
            results.append(result)
    
    return results

def normalize_probability_densities(results: List[Dict]) -> List[Dict]:
    """확률 밀도를 가장 큰 max_turns에 맞춰 정규화"""
    # 모든 결과에서 최대 턴 수 찾기
    global_max_turns = max(result['max_turns'] for result in results)
    
    normalized_results = []
    for result in results:
        normalized_result = result.copy()
        
        # 부족한 턴에 대해 0.0으로 채우기
        for turn in range(1, global_max_turns + 1):
            turn_key = f"turn_{turn}"
            if turn_key not in normalized_result:
                normalized_result[turn_key] = 0.0
        
        normalized_results.append(normalized_result)
    
    return normalized_results

def results_to_dataframe(results: List[Dict]) -> pd.DataFrame:
    """결과를 DataFrame으로 변환"""
    # 정규화된 결과 생성
    normalized_results = normalize_probability_densities(results)
    
    # DataFrame 생성
    df = pd.DataFrame(normalized_results)
    
    # 열 순서 정렬 (기본 정보 + turn_1, turn_2, ...)
    basic_cols = ['n_pairs', 'memory_size', 'num_simulations', 'min_turns', 
                  'max_turns', 'average_turns', 'std_turns', 'success_rate']
    
    turn_cols = [col for col in df.columns if col.startswith('turn_')]
    turn_cols.sort(key=lambda x: int(x.split('_')[1]))  # turn_1, turn_2, ... 순서로 정렬
    
    column_order = basic_cols + turn_cols
    df = df[column_order]
    
    return df

def print_summary_statistics(results: List[Dict]):
    """요약 통계 출력"""
    print("\n=== 시뮬레이션 결과 요약 ===")
    print("=" * 80)
    
    for result in results:
        n_pairs = result['n_pairs']
        memory_size = result['memory_size']
        avg_turns = result['average_turns']
        std_turns = result['std_turns']
        min_turns = result['min_turns']
        max_turns = result['max_turns']
        success_rate = result['success_rate']
        
        print(f"\n【{n_pairs}쌍, 메모리 {memory_size}개】")
        print(f"  평균 턴 수: {avg_turns:.3f} ± {std_turns:.3f}")
        print(f"  범위: {min_turns} ~ {max_turns} 턴")
        print(f"  성공률: {success_rate:.1%}")
        
        # 확률이 높은 상위 3개 턴 출력
        turn_probs = [(k, v) for k, v in result.items() if k.startswith('turn_') and v > 0]
        turn_probs.sort(key=lambda x: x[1], reverse=True)
        
        print("  가장 빈번한 턴 수:")
        for turn_key, prob in turn_probs[:3]:
            turn_num = turn_key.split('_')[1]
            print(f"    {turn_num}턴: {prob:.1%}")
def extract_probability_density_dataframe(results: List[Dict]) -> pd.DataFrame:
    """인덱스: (n_pairs, memory_size), 컬럼: turn_1 ~ turn_n 형태의 DataFrame 추출"""
    # 정규화된 결과 생성
    normalized_results = normalize_probability_densities(results)
    
    # DataFrame 생성
    df = pd.DataFrame(normalized_results)
    
    # turn 컬럼들만 추출 및 정렬
    turn_cols = [col for col in df.columns if col.startswith('turn_')]
    turn_cols.sort(key=lambda x: int(x.split('_')[1]))
    
    # 필요한 컬럼들만 선택
    probability_df = df[['n_pairs', 'memory_size'] + turn_cols].copy()
    
    # 인덱스 설정
    probability_df.set_index(['n_pairs', 'memory_size'], inplace=True)
    
    return probability_df
# 실행 예제
if __name__ == "__main__":
    print("=== 상세 결과 카드 게임 시뮬레이션 ===\n")
    
    # 시뮬레이션 설정
    n_pairs_list = [2, 3, 4]  # 카드 쌍 수
    memory_sizes = [1, 2, 3, 5]  # 메모리 크기
    num_simulations = 5000  # 시뮬레이션 횟수
    
    # 배치 시뮬레이션 실행
    results = run_batch_detailed_simulation(n_pairs_list, memory_sizes, num_simulations)
    
    # 요약 통계 출력
    print_summary_statistics(results)
    
    # DataFrame으로 변환
    df = results_to_dataframe(results)
    
    print(f"\n=== DataFrame 형태 결과 ===")
    print("형태:", df.shape)
    print("\n기본 정보 열:")
    basic_cols = ['n_pairs', 'memory_size', 'average_turns', 'min_turns', 'max_turns', 'success_rate']
    print(df[basic_cols].to_string(index=False))
    
    print(f"\n=== 확률 밀도 예시 (첫 번째 결과) ===")
    first_result = results[0]
    print(f"{first_result['n_pairs']}쌍, 메모리 {first_result['memory_size']}개:")
    
    turn_cols = [k for k in first_result.keys() if k.startswith('turn_')]
    turn_cols.sort(key=lambda x: int(x.split('_')[1]))
    
    for turn_key in turn_cols[:10]:  # 처음 10턴만 출력
        turn_num = turn_key.split('_')[1]
        prob = first_result[turn_key]
        if prob > 0:
            print(f"  {turn_num}턴: {prob:.4f} ({prob:.1%})")
    
    # 이론적 완벽 플레이어와 비교
    print(f"\n=== 이론적 완벽 플레이어 비교 ===")
    for n_pairs in n_pairs_list:
        theoretical_perfect = 1.61 * n_pairs
        print(f"\n{n_pairs}쌍:")
        print(f"  이론적 완벽 플레이어: {theoretical_perfect:.2f}턴")
        
        # 해당 쌍에 대한 결과들 출력
        pair_results = [r for r in results if r['n_pairs'] == n_pairs]
        for result in pair_results:
            memory_size = result['memory_size']
            avg_turns = result['average_turns']
            diff = avg_turns - theoretical_perfect
            print(f"  메모리 {memory_size}개: {avg_turns:.2f}턴 (차이: {diff:+.2f})")
    
    # 전체 결과 반환을 위한 함수도 제공
    def get_simulation_results():
        """시뮬레이션 결과 반환 (다른 스크립트에서 사용 가능)"""
        return results, df, prob_df

=== 상세 결과 카드 게임 시뮬레이션 ===

시뮬레이션 중: 2쌍, 메모리 크기 1...
시뮬레이션 중: 2쌍, 메모리 크기 2...
시뮬레이션 중: 2쌍, 메모리 크기 3...
시뮬레이션 중: 2쌍, 메모리 크기 5...
시뮬레이션 중: 3쌍, 메모리 크기 1...
시뮬레이션 중: 3쌍, 메모리 크기 2...
시뮬레이션 중: 3쌍, 메모리 크기 3...
시뮬레이션 중: 3쌍, 메모리 크기 5...
시뮬레이션 중: 4쌍, 메모리 크기 1...
시뮬레이션 중: 4쌍, 메모리 크기 2...
시뮬레이션 중: 4쌍, 메모리 크기 3...
시뮬레이션 중: 4쌍, 메모리 크기 5...

=== 시뮬레이션 결과 요약 ===

【2쌍, 메모리 1개】
  평균 턴 수: 4.002 ± 2.453
  범위: 2 ~ 20 턴
  성공률: 100.0%
  가장 빈번한 턴 수:
    2턴: 33.6%
    3턴: 21.5%
    4턴: 15.1%

【2쌍, 메모리 2개】
  평균 턴 수: 3.329 ± 1.498
  범위: 2 ~ 17 턴
  성공률: 100.0%
  가장 빈번한 턴 수:
    2턴: 34.1%
    3턴: 31.8%
    4턴: 17.4%

【2쌍, 메모리 3개】
  평균 턴 수: 2.980 ± 0.905
  범위: 2 ~ 9 턴
  성공률: 100.0%
  가장 빈번한 턴 수:
    3턴: 44.7%
    2턴: 32.4%
    4턴: 17.2%

【2쌍, 메모리 5개】
  평균 턴 수: 2.952 ± 0.881
  범위: 2 ~ 7 턴
  성공률: 100.0%
  가장 빈번한 턴 수:
    3턴: 45.2%
    2턴: 33.1%
    4턴: 16.3%

【3쌍, 메모리 1개】
  평균 턴 수: 8.901 ± 5.064
  범위: 3 ~ 47 턴
  성공률: 100.0%
  가장 빈번한 턴 수:
    5턴: 10.7%
    4턴: 10.6%
    6턴: 10.4%

【3쌍, 메모리 2개】
  평균 턴 수: 6.687 ± 2.903
 

In [110]:
results = run_batch_detailed_simulation([2, 3, 6,8,10], [1, 2, 3, 4,5,6,7,8,9,10,11,12], 20000)

# 요청하신 형태의 DataFrame 추출
prob_df = extract_probability_density_dataframe(results)
prob_df

시뮬레이션 중: 2쌍, 메모리 크기 1...
시뮬레이션 중: 2쌍, 메모리 크기 2...
시뮬레이션 중: 2쌍, 메모리 크기 3...
시뮬레이션 중: 2쌍, 메모리 크기 4...
시뮬레이션 중: 2쌍, 메모리 크기 5...
시뮬레이션 중: 2쌍, 메모리 크기 6...
시뮬레이션 중: 2쌍, 메모리 크기 7...
시뮬레이션 중: 2쌍, 메모리 크기 8...
시뮬레이션 중: 2쌍, 메모리 크기 9...
시뮬레이션 중: 2쌍, 메모리 크기 10...
시뮬레이션 중: 2쌍, 메모리 크기 11...
시뮬레이션 중: 2쌍, 메모리 크기 12...
시뮬레이션 중: 3쌍, 메모리 크기 1...
시뮬레이션 중: 3쌍, 메모리 크기 2...
시뮬레이션 중: 3쌍, 메모리 크기 3...
시뮬레이션 중: 3쌍, 메모리 크기 4...
시뮬레이션 중: 3쌍, 메모리 크기 5...
시뮬레이션 중: 3쌍, 메모리 크기 6...
시뮬레이션 중: 3쌍, 메모리 크기 7...
시뮬레이션 중: 3쌍, 메모리 크기 8...
시뮬레이션 중: 3쌍, 메모리 크기 9...
시뮬레이션 중: 3쌍, 메모리 크기 10...
시뮬레이션 중: 3쌍, 메모리 크기 11...
시뮬레이션 중: 3쌍, 메모리 크기 12...
시뮬레이션 중: 6쌍, 메모리 크기 1...
시뮬레이션 중: 6쌍, 메모리 크기 2...
시뮬레이션 중: 6쌍, 메모리 크기 3...
시뮬레이션 중: 6쌍, 메모리 크기 4...
시뮬레이션 중: 6쌍, 메모리 크기 5...
시뮬레이션 중: 6쌍, 메모리 크기 6...
시뮬레이션 중: 6쌍, 메모리 크기 7...
시뮬레이션 중: 6쌍, 메모리 크기 8...
시뮬레이션 중: 6쌍, 메모리 크기 9...
시뮬레이션 중: 6쌍, 메모리 크기 10...
시뮬레이션 중: 6쌍, 메모리 크기 11...
시뮬레이션 중: 6쌍, 메모리 크기 12...
시뮬레이션 중: 8쌍, 메모리 크기 1...
시뮬레이션 중: 8쌍, 메모리 크기 2...
시뮬레이션 중: 8쌍, 메모리 크기 3...
시뮬레이션 중: 8쌍, 메모리

turn_1   turn_2   turn_3   turn_4   turn_5   turn_6  \
n_pairs memory_size                                                        
2       1               0.0  0.33245  0.22360  0.14785  0.10080  0.06370   
        2               0.0  0.33370  0.32720  0.16950  0.08545  0.04185   
        3               0.0  0.33640  0.44370  0.16780  0.04050  0.00940   
        4               0.0  0.32825  0.45065  0.16600  0.04360  0.00915   
        5               0.0  0.33635  0.44060  0.16430  0.04555  0.01045   
        6               0.0  0.33515  0.44625  0.16305  0.04145  0.01120   
        7               0.0  0.33265  0.44975  0.16465  0.04170  0.00955   
        8               0.0  0.33560  0.44150  0.16880  0.04100  0.01025   
        9               0.0  0.33450  0.44185  0.16780  0.04355  0.00985   
        10              0.0  0.33625  0.44095  0.16715  0.04380  0.00915   
        11              0.0  0.33400  0.45085  0.16255  0.04150  0.00870   
        12              0.0  0.33585  0.44320  0.16340  0.04615  0.00890   
3       1               0.0  0.00000  0.06830  0.09870  0.10855  0.10620   
        2               0.0  0.00000  0.06720  0.15350  0.17910  0.16265   
        3               0.0  0.00000  0.06675  0.27980  0.27330  0.17680   
        4               0.0  0.00000  0.06725  0.28225  0.32795  0.18665   
        5               0.0  0.00000  0.06350  0.28105  0.33965  0.20025   
        6               0.0  0.00000  0.06390  0.28900  0.33235  0.19465   
        7               0.0  0.00000  0.06490  0.28570  0.33595  0.19950   
        8               0.0  0.00000  0.06515  0.28560  0.33535  0.19430   
        9               0.0  0.00000  0.06480  0.28305  0.34085  0.19405   
        10              0.0  0.00000  0.06190  0.28155  0.33975  0.19790   
        11              0.0  0.00000  0.06600  0.28660  0.34115  0.18865   
        12              0.0  0.00000  0.06775  0.28310  0.34420  0.19150   
6       1               0.0  0.00000  0.00000  0.00000  0.00000  0.00020   
        2               0.0  0.00000  0.00000  0.00000  0.00000  0.00000   
        3               0.0  0.00000  0.00000  0.00000  0.00000  0.00005   
        4               0.0  0.00000  0.00000  0.00000  0.00000  0.00020   
        5               0.0  0.00000  0.00000  0.00000  0.00000  0.00015   
        6               0.0  0.00000  0.00000  0.00000  0.00000  0.00010   
        7               0.0  0.00000  0.00000  0.00000  0.00000  0.00010   
        8               0.0  0.00000  0.00000  0.00000  0.00000  0.00030   
        9               0.0  0.00000  0.00000  0.00000  0.00000  0.00010   
        10              0.0  0.00000  0.00000  0.00000  0.00000  0.00005   
        11              0.0  0.00000  0.00000  0.00000  0.00000  0.00005   
        12              0.0  0.00000  0.00000  0.00000  0.00000  0.00010   
8       1               0.0  0.00000  0.00000  0.00000  0.00000  0.00000   
        2               0.0  0.00000  0.00000  0.00000  0.00000  0.00000   
        3               0.0  0.00000  0.00000  0.00000  0.00000  0.00000   
        4               0.0  0.00000  0.00000  0.00000  0.00000  0.00000   
        5               0.0  0.00000  0.00000  0.00000  0.00000  0.00000   
        6               0.0  0.00000  0.00000  0.00000  0.00000  0.00000   
        7               0.0  0.00000  0.00000  0.00000  0.00000  0.00000   
        8               0.0  0.00000  0.00000  0.00000  0.00000  0.00000   
        9               0.0  0.00000  0.00000  0.00000  0.00000  0.00000   
        10              0.0  0.00000  0.00000  0.00000  0.00000  0.00000   
        11              0.0  0.00000  0.00000  0.00000  0.00000  0.00000   
        12              0.0  0.00000  0.00000  0.00000  0.00000  0.00000   
10      1               0.0  0.00000  0.00000  0.00000  0.00000  0.00000   
        2               0.0  0.00000  0.00000  0.00000  0.00000  0.00000   
        3               0.0  0.00000  0.00000  0.00000  0.00000  0.00

In [111]:
import pandas as pd
import numpy as np
import random
from scipy import stats
from typing import Dict, List, Tuple
from collections import defaultdict

class GameLevel:
    """게임 레벨 정보"""
    def __init__(self, level: int, n_pairs: int, time_limit: float, 
                 success_reward: int, fail_reward: int = 1, 
                 turn_duration: float = 0.88, trick_cards: int = 0):
        self.level = level
        self.n_pairs = n_pairs
        self.time_limit = time_limit
        self.success_reward = success_reward
        self.fail_reward = fail_reward
        self.turn_duration = turn_duration
        self.trick_cards = trick_cards  # 트릭카드 개수
        self.basic_cards = n_pairs - trick_cards  # 기본카드 개수
        
        # 트릭카드로 인한 체감 난이도 증가를 turn_duration에 반영
        if trick_cards > 0:
            # 트릭카드 1개당 20% 속도 감소 (더 신중하게 플레이)
            self.effective_turn_duration = turn_duration * (1 + trick_cards * 0.1)
        elif level < 4:
            self.effective_turn_duration = turn_duration * (0.6)
        else:
            self.effective_turn_duration = turn_duration

class GameSimulator:
    """완전한 게임 시뮬레이션 엔진"""
    
    def __init__(self, 
                 probability_df: pd.DataFrame,
                 game_levels: Dict = None,
                 user_memory_distribution: Dict = None,
                 max_retries: int = 8,
                 retry_probability: float = 0.5,
                 turn_time_params: Dict = None):
        """
        초기화
        
        Args:
            probability_df: extract_probability_density_dataframe()으로 생성된 확률 분포 DataFrame
            game_levels: 게임 레벨 설정 (None이면 기본값 사용)
            user_memory_distribution: 유저 메모리 분포 (None이면 기본값 사용)
            max_retries: 최대 재시도 횟수
            retry_probability: 재시도 확률
            turn_time_params: 턴 시간 파라미터
        """
        self.prob_df = probability_df
        self.game_levels = game_levels or self._get_default_game_levels()
        self.user_memory_distribution = user_memory_distribution or self._get_default_memory_distribution()
        self.max_retries = max_retries
        self.retry_probability = retry_probability
        self.turn_time_params = turn_time_params or self._get_default_turn_time_params()
    
    def _get_default_game_levels(self) -> Dict:
        """기본 게임 레벨 설정"""
        return {
            1: GameLevel(1, 2, 5.0, 2, 1, 0.8, 0),    # 트릭카드 없음
            2: GameLevel(2, 3, 7.0, 2, 1, 0.9, 0),    # 트릭카드 없음
            3: GameLevel(3, 6, 9.0, 2, 1, 1.0, 0),    # 트릭카드 없음
            4: GameLevel(4, 8, 13.0, 2, 1, 1.1, 0),   # 트릭카드 없음
            5: GameLevel(5, 8, 15.0, 3, 1, 1.2, 2),   # 트릭카드 2개 → 1.2 * 1.4 = 1.68초
            6: GameLevel(6, 10, 21.0, 3, 1, 1.3, 0),  # 트릭카드 없음
            7: GameLevel(7, 10, 25.0, 6, 1, 1.4, 2)   # 트릭카드 2개 → 1.4 * 1.4 = 1.96초
        }
    
    def _get_default_memory_distribution(self) -> Dict:
        """기본 유저 메모리 분포"""
        return {
            1: 0.02,
            2: 0.05,
            3: 0.08,
            4: 0.15,
            5: 0.245,   # 중앙(최대)
            6: 0.19,
            7: 0.13,
            8: 0.075,
            9: 0.04,
            10: 0.02
        }

    
    def _get_default_turn_time_params(self) -> Dict:
        """기본 턴 시간 파라미터"""
        return {
            'distribution': 't',
            'df': 10,
            'loc': 0.88,
            'scale': 1,
            'min_time': 0.1
        }
    
    def sample_user_memory(self) -> int:
        """유저 메모리 크기 샘플링"""
        memory_sizes = list(self.user_memory_distribution.keys())
        probabilities = list(self.user_memory_distribution.values())
        return np.random.choice(memory_sizes, p=probabilities)
    
    def sample_turn_count(self, n_pairs: int, memory_size: int) -> int:
        """확률 분포표에서 턴 수 샘플링"""
        try:
            # MultiIndex 확인
            if isinstance(self.prob_df.index, pd.MultiIndex):
                # MultiIndex인 경우 (n_pairs, memory_size)
                available_memories = self.prob_df.index.get_level_values(1).unique()
                if memory_size not in available_memories:
                    memory_size = min(available_memories, key=lambda x: abs(x - memory_size))
                
                prob_row = self.prob_df.loc[(n_pairs, memory_size)]
            else:
                # 단일 인덱스인 경우 - memory_size만 사용
                if memory_size not in self.prob_df.index:
                    available_memories = self.prob_df.index.unique()
                    memory_size = min(available_memories, key=lambda x: abs(x - memory_size))
                
                prob_row = self.prob_df.loc[memory_size]
            
            # 0이 아닌 확률들만 추출
            non_zero_probs = prob_row[prob_row > 0]
            
            if len(non_zero_probs) == 0:
                # 확률이 모두 0인 경우 기본값 반환
                return n_pairs
            
            # 확률 정규화
            probabilities = non_zero_probs.values
            probabilities = probabilities / np.sum(probabilities)
            
            # 턴 번호 추출 (컬럼명에서 숫자 부분만 추출)
            turn_numbers = []
            for col in non_zero_probs.index:
                if str(col).startswith('turn_'):
                    turn_numbers.append(int(col.split('_')[1]))
                else:
                    turn_numbers.append(int(col))
            
            # 샘플링
            sampled_turn = np.random.choice(turn_numbers, p=probabilities)
            return sampled_turn
            
        except (KeyError, IndexError) as e:
            print(f"경고: ({n_pairs}쌍, 메모리{memory_size}) 조건이 없어서 기본값 {n_pairs} 사용. 에러: {e}")
            return n_pairs
    
    def sample_turn_time(self, turn_count: int, game_level: GameLevel = None) -> float:
        """턴당 소요 시간 샘플링 (트릭카드 영향 반영)"""
        params = self.turn_time_params
        
        # 기본 턴 지속시간
        base_turn_duration = params['loc']
        
        # 트릭카드로 인한 체감 난이도 반영
        if game_level and hasattr(game_level, 'trick_cards') and game_level.trick_cards > 0:
            # 트릭카드 1개당 20% 속도 감소 (더 신중하게 플레이)
            difficulty_multiplier = 1 + (game_level.trick_cards * 0.2)
            effective_turn_duration = base_turn_duration * difficulty_multiplier
        else:
            effective_turn_duration = base_turn_duration
        
        # 분포에 따른 샘플링
        if params['distribution'] == 't':
            # T분포 샘플링
            turn_times = stats.t.rvs(
                df=params['df'], 
                loc=effective_turn_duration,
                scale=params['scale'], 
                size=turn_count
            )
        elif params['distribution'] == 'normal':
            # 정규분포 샘플링
            turn_times = np.random.normal(
                loc=effective_turn_duration,
                scale=params['scale'], 
                size=turn_count
            )
        elif params['distribution'] == 'uniform':
            # 균등분포 샘플링
            turn_times = np.random.uniform(
                low=effective_turn_duration - params['scale'],
                high=effective_turn_duration + params['scale'],
                size=turn_count
            )
        else:
            # 기본값: 고정 시간
            turn_times = np.full(turn_count, effective_turn_duration)
        
        # 음수 시간 제거 (최소 시간 적용)
        turn_times = np.maximum(turn_times, params['min_time'])
        
        return np.sum(turn_times)
    
    def should_retry(self) -> bool:
        """재시도 여부 결정"""
        return np.random.uniform(0, 1) < self.retry_probability
    
    def simulate_level(self, level: int, memory_size: int) -> Dict:
        """단일 레벨 시뮬레이션 - 재시도와 이탈을 구분"""
        game_level = self.game_levels[level]
        attempts = 0
        successes = 0      # 성공한 시도 수
        failures = 0       # 실패한 시도 수
        retries = 0        # 재시도한 횟수 (광고 시청)
        abandons = 0       # 이탈한 횟수 (광고 시청 안함)
        total_cost = 0     # 우리가 유저에게 지출한 게임 리워드 비용
        total_revenue = 0  # 유저가 재시도하여 광고 시청 매출
        
        for attempt in range(self.max_retries + 1):  # 최초 시도 + 재시도
            attempts += 1
            
            # 턴 수 샘플링
            turn_count = self.sample_turn_count(game_level.n_pairs, memory_size)
            
            # 소요 시간 샘플링 (트릭카드 영향 반영)
            total_time = self.sample_turn_time(turn_count, game_level)
            
            # 성공/실패 판정
            if total_time <= game_level.time_limit:
                # 성공 - 유저에게 리워드 지급 (우리 비용)
                successes += 1
                total_cost = game_level.success_reward
                break  # 성공하면 레벨 완료
            else:
                # 실패
                failures += 1
                
                # 마지막 시도가 아니고 재시도 의사가 있는 경우
                if attempt < self.max_retries and self.should_retry():
                    # 재시도 = 광고 시청 = 매출 발생
                    retries += 1
                    total_revenue += game_level.fail_reward
                    continue
                else:
                    # 마지막 시도 실패 = 이탈 = 매출 없음
                    abandons += 1
                    break
        
        level_success = successes > 0  # 레벨 완료 여부
        
        return {
            'level': level,
            'memory_size': memory_size,
            'attempts': attempts,
            'successes': successes,
            'failures': failures,
            'retries': retries,        # 재시도 횟수 (매출 발생)
            'abandons': abandons,      # 이탈 횟수 (매출 없음)
            'level_success': level_success,
            'success': level_success,
            'total_cost': total_cost,
            'total_revenue': total_revenue,
            'net_profit': total_revenue - total_cost,
            'n_pairs': game_level.n_pairs,
            'time_limit': game_level.time_limit,
            'turn_duration': game_level.turn_duration,
            'trick_cards': game_level.trick_cards
        }
    
    def simulate_full_game(self, memory_size: int) -> Dict:
        """전체 게임 시뮬레이션 - 재시도와 이탈 구분"""
        results = []
        current_level = 1
        total_stats = {
            'total_attempts': 0,
            'total_successes': 0,  # 성공한 시도 수
            'total_failures': 0,   # 실패한 시도 수
            'total_retries': 0,    # 재시도 횟수 (매출 발생)
            'total_abandons': 0,   # 이탈 횟수 (매출 없음)
            'total_cost': 0,
            'total_revenue': 0,
            'levels_completed': 0,
            'final_level': 0
        }
        
        while current_level <= len(self.game_levels):
            level_result = self.simulate_level(current_level, memory_size)
            results.append(level_result)
            
            # 시도 단위로 누적
            total_stats['total_attempts'] += level_result['attempts']
            total_stats['total_successes'] += level_result['successes']
            total_stats['total_failures'] += level_result['failures']
            total_stats['total_retries'] += level_result['retries']
            total_stats['total_abandons'] += level_result['abandons']
            total_stats['total_cost'] += level_result['total_cost']
            total_stats['total_revenue'] += level_result['total_revenue']
            total_stats['final_level'] = current_level
            
            if level_result['level_success']:  # 레벨 완료 여부 확인
                total_stats['levels_completed'] += 1
                current_level += 1
            else:
                break  # 레벨 실패로 게임 종료
        
        total_stats['net_profit'] = total_stats['total_revenue'] - total_stats['total_cost']
        
        return {
            'memory_size': memory_size,
            'level_results': results,
            'total_stats': total_stats
        }
    
    def run_simulation(self, num_users: int = 10000) -> Tuple[pd.DataFrame, pd.DataFrame]:
        """대규모 시뮬레이션 실행"""
        print(f"=== {num_users}명 유저 게임 시뮬레이션 시작 ===")
        
        user_results = []
        level_stats = defaultdict(lambda: {
            'attempts': 0, 'successes': 0, 'failures': 0, 
            'retries': 0, 'abandons': 0,  # 재시도와 이탈 구분
            'total_cost': 0, 'total_revenue': 0
        })
        
        for user_id in range(num_users):
            if (user_id + 1) % 1000 == 0:
                print(f"진행률: {user_id + 1}/{num_users} ({(user_id + 1)/num_users*100:.1f}%)")
            
            memory_size = self.sample_user_memory()
            game_result = self.simulate_full_game(memory_size)
            user_results.append(game_result)
            
            for level_result in game_result['level_results']:
                level = level_result['level']
                
                # 시도 단위로 누적 (재시도와 이탈 구분)
                level_stats[level]['attempts'] += level_result['attempts']
                level_stats[level]['successes'] += level_result['successes']
                level_stats[level]['failures'] += level_result['failures']
                level_stats[level]['retries'] += level_result['retries']      # 재시도 횟수
                level_stats[level]['abandons'] += level_result['abandons']    # 이탈 횟수
                level_stats[level]['total_cost'] += level_result['total_cost']
                level_stats[level]['total_revenue'] += level_result['total_revenue']
        
        user_df = self._create_user_dataframe(user_results)
        level_df = self._create_level_dataframe(level_stats)
        
        return user_df, level_df
    
    def _create_user_dataframe(self, user_results: List[Dict]) -> pd.DataFrame:
        """유저 결과 DataFrame 생성 - 재시도 횟수 추가"""
        rows = []
        for i, result in enumerate(user_results):
            total_stats = result['total_stats']
            
            # 재시도 횟수 계산
            retry_count = total_stats['total_attempts'] - total_stats['total_successes']
            
            row = {
                'user_id': i + 1,
                'memory_size': result['memory_size'],
                **total_stats,
                'retry_count': retry_count  # 재시도 횟수 추가
            }
            rows.append(row)
        
        return pd.DataFrame(rows)
        
    def _create_level_dataframe(self, level_stats: Dict) -> pd.DataFrame:
        """레벨별 통계 DataFrame 생성 - 재시도와 이탈 구분"""
        rows = []
        for level in range(1, len(self.game_levels) + 1):
            stats = level_stats[level]
            game_level = self.game_levels[level]
            
            # 재시도와 이탈 횟수 (이미 구분되어 있음)
            retry_count = stats['retries']
            abandon_count = stats['abandons']
            
            total_plays = stats['successes'] + stats['failures']
            success_rate = stats['successes'] / total_plays if total_plays > 0 else 0
            avg_attempts = stats['attempts'] / total_plays if total_plays > 0 else 0
            avg_retries = retry_count / total_plays if total_plays > 0 else 0
            avg_abandons = abandon_count / total_plays if total_plays > 0 else 0
            net_profit = stats['total_revenue'] - stats['total_cost']
            
            row = {
                'level': level,
                'n_pairs': game_level.n_pairs,
                'time_limit': game_level.time_limit,
                'success_reward': game_level.success_reward,
                'fail_reward': game_level.fail_reward,
                'turn_duration': game_level.turn_duration,
                'trick_cards': game_level.trick_cards,
                'total_plays': total_plays,
                'attempts': stats['attempts'],
                'successes': stats['successes'],
                'failures': stats['failures'],
                'retry_count': retry_count,      # 재시도 횟수 (매출 발생)
                'abandon_count': abandon_count,  # 이탈 횟수 (매출 없음)
                'success_rate': success_rate,
                'avg_attempts_per_play': avg_attempts,
                'avg_retries_per_play': avg_retries,
                'avg_abandons_per_play': avg_abandons,
                'total_cost': stats['total_cost'],
                'total_revenue': stats['total_revenue'],
                'net_profit': net_profit,
                'profit_per_play': net_profit / total_plays if total_plays > 0 else 0
            }
            rows.append(row)
        
        return pd.DataFrame(rows)

def print_simulation_summary(user_df: pd.DataFrame, level_df: pd.DataFrame):
    """시뮬레이션 결과 요약 출력"""
    print("\n" + "="*80)
    print("🎮 게임 시뮬레이션 결과 요약")
    print("="*80)
    
    total_users = len(user_df)
    total_revenue = user_df['total_revenue'].sum()
    total_cost = user_df['total_cost'].sum()
    net_profit = total_revenue - total_cost
    avg_levels = user_df['levels_completed'].mean()
    
    print(f"👥 전체 사용자: {total_users:,}명")
    print(f"💰 총 수익: {total_revenue:,.0f}원")
    print(f" 총 비용: {total_cost:,.0f}원")
    print(f"📊 순이익: {net_profit:,.0f}원")
    print(f"🎯 평균 완료 레벨: {avg_levels:.1f}단계")
    
    print(f"\n📈 레벨별 통계:")
    level_summary = level_df[['level', 'n_pairs', 'trick_cards', 'time_limit', 'successes', 'failures', 
                             'success_rate', 'total_cost', 'total_revenue', 'net_profit']].round(2)
    print(level_summary.to_string(index=False))
    
    print(f"\n🧠 메모리 크기별 성과:")
    memory_stats = user_df.groupby('memory_size').agg({
        'levels_completed': 'mean',
        'total_revenue': 'mean',
        'total_cost': 'mean',
        'net_profit': 'mean'
    }).round(2)
    
    print(memory_stats)

In [112]:
# 1. memory_v2에서 생성된 prob_df 사용 (이미 생성되어 있다고 가정)
# prob_df = extract_probability_density_dataframe(results)

# 2. 게임 시뮬레이터 생성
simulator = GameSimulator(prob_df)

# 3. 시뮬레이션 실행
user_df, level_df = simulator.run_simulation(num_users=22000)

# 4. 결과 출력
print_simulation_summary(user_df, level_df)

# 5. 트릭카드 영향 확인
print("\n" + "="*80)
print("트릭카드 영향 분석")
print("="*80)

# 레벨별 평균 소요 시간 확인
for level in [1, 2, 3, 4, 5, 6, 7]:
    game_level = simulator.game_levels[level]
    print(f"Level {level}: {game_level.n_pairs}쌍, 트릭카드 {game_level.trick_cards}개")
    print(f"  기본 턴 지속시간: {game_level.turn_duration:.2f}초")
    print(f"  효과적 턴 지속시간: {game_level.effective_turn_duration:.2f}초")
    print(f"  시간 증가율: {(game_level.effective_turn_duration / game_level.turn_duration - 1) * 100:.1f}%")
    print()

# 6. 결과 저장 (선택사항)
# user_df.to_csv('user_results.csv', index=False)
# level_df.to_csv('level_results.csv', index=False)

=== 22000명 유저 게임 시뮬레이션 시작 ===
진행률: 1000/22000 (4.5%)
진행률: 2000/22000 (9.1%)
진행률: 3000/22000 (13.6%)
진행률: 4000/22000 (18.2%)
진행률: 5000/22000 (22.7%)
진행률: 6000/22000 (27.3%)
진행률: 7000/22000 (31.8%)
진행률: 8000/22000 (36.4%)
진행률: 9000/22000 (40.9%)
진행률: 10000/22000 (45.5%)
진행률: 11000/22000 (50.0%)
진행률: 12000/22000 (54.5%)
진행률: 13000/22000 (59.1%)
진행률: 14000/22000 (63.6%)
진행률: 15000/22000 (68.2%)
진행률: 16000/22000 (72.7%)
진행률: 17000/22000 (77.3%)
진행률: 18000/22000 (81.8%)
진행률: 19000/22000 (86.4%)
진행률: 20000/22000 (90.9%)
진행률: 21000/22000 (95.5%)
진행률: 22000/22000 (100.0%)

🎮 게임 시뮬레이션 결과 요약
👥 전체 사용자: 22,000명
💰 총 수익: 22,555원
 총 비용: 88,973원
📊 순이익: -66,418원
🎯 평균 완료 레벨: 2.0단계

📈 레벨별 통계:
 level  n_pairs  trick_cards  time_limit  successes  failures  success_rate  total_cost  total_revenue  net_profit
     1        2            0         5.0      20276      3404          0.86       40552           1680      -38872
     2        3            0         7.0      17580      5453          0.76       35160 

In [62]:
user_df.total_attempts.value_counts()

total_attempts
3     6370
4     4809
5     2877
2     2314
6     1745
1     1598
7      979
8      547
9      336
10     190
11     107
12      61
13      29
14      19
15      11
17       4
16       3
18       1
Name: count, dtype: int64

In [100]:
level_df

,level,n_pairs,time_limit,success_reward,fail_reward,turn_duration,trick_cards,total_plays,attempts,successes,...,retry_count,abandon_count,success_rate,avg_attempts_per_play,avg_retries_per_play,avg_abandons_per_play,total_cost,total_revenue,net_profit,profit_per_play
0,1,2,5.0,2,1,0.8,0,23669,23669,20297,...,1669,1703,0.857535,1.0,0.070514,0.071951,40594,1669,-38925,-1.644556
1,2,3,7.0,2,1,0.9,0,23059,23059,17591,...,2762,2706,0.762869,1.0,0.119780,0.117351,35182,2762,-32420,-1.405959
2,3,6,9.0,2,1,1.0,0,30111,30111,4871,...,12520,12720,0.161768,1.0,0.415795,0.422437,9742,12520,2778,0.092259
3,4,8,13.0,2,1,1.1,0,8545,8545,1200,...,3674,3671,0.140433,1.0,0.429959,0.429608,2400,3674,1274,0.149093
4,5,8,15.0,3,1,1.2,2,2212,2212,158,...,1012,1042,0.071429,1.0,0.457505,0.471067,474,1012,538,0.243219
5,6,10,21.0,3,1,1.3,0,227,227,78,...,69,80,0.343612,1.0,0.303965,0.352423,234,69,-165,-0.726872
6,7,10,25.0,6,1,1.4,2,117,117,30,...,39,48,0.256410,1.0,0.333333,0.410256,180,39,-141,-1.205128


In [95]:
level_df[['attempts', 'successes','failures' ,'abandon_count', 'retry_count']]

,attempts,successes,failures,abandon_count,retry_count
0,23669,20297,3372,1703,1669
1,23059,17591,5468,2706,2762
2,30111,4871,25240,12720,12520
3,8545,1200,7345,3671,3674
4,2212,158,2054,1042,1012
5,227,78,149,80,69
6,117,30,87,48,39


In [102]:
user_df.colum

,user_id,memory_size,total_attempts,total_successes,total_failures,total_retries,total_abandons,total_cost,total_revenue,levels_completed,final_level,net_profit,retry_count
0,1,4,5,2,3,2,1,4,2,2,3,-2,3
1,2,6,2,1,1,0,1,2,0,1,2,-2,1
2,3,6,1,0,1,0,1,0,0,0,1,0,1
3,4,1,2,1,1,0,1,2,0,1,2,-2,1
4,5,6,3,2,1,0,1,4,0,2,3,-4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21995,21996,6,7,3,4,3,1,6,3,3,4,-3,4
21996,21997,3,7,2,5,4,1,4,4,2,3,0,5
21997,21998,5,3,2,1,0,1,4,0,2,3,-4,1
21998,21999,7,3,2,1,0,1,4,0,2,3,-4,1


In [101]:
user_df.groupby('levels_completed')[['total_attempts', 'total_successes', 'total_failures', 'total_abandons', 'total_retries']].sum()

,total_attempts,total_successes,total_failures,total_abandons,total_retries
levels_completed,,,,,
0,1838,0,1838,1703,135
1,6038,2706,3332,2706,626
2,50055,25440,24615,12720,11895
3,20766,11013,9753,3671,6082
4,7758,4168,3590,1042,2548
5,697,400,297,80,217
6,494,288,206,48,158
7,294,210,84,0,84


In [91]:
user_df

,user_id,memory_size,total_attempts,total_successes,total_failures,total_retries,total_abandons,total_cost,total_revenue,levels_completed,final_level,net_profit,retry_count
0,1,4,5,2,3,2,1,4,2,2,3,-2,3
1,2,6,2,1,1,0,1,2,0,1,2,-2,1
2,3,6,1,0,1,0,1,0,0,0,1,0,1
3,4,1,2,1,1,0,1,2,0,1,2,-2,1
4,5,6,3,2,1,0,1,4,0,2,3,-4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21995,21996,6,7,3,4,3,1,6,3,3,4,-3,4
21996,21997,3,7,2,5,4,1,4,4,2,3,0,5
21997,21998,5,3,2,1,0,1,4,0,2,3,-4,1
21998,21999,7,3,2,1,0,1,4,0,2,3,-4,1


In [68]:
level_df

,level,n_pairs,time_limit,success_reward,fail_reward,turn_duration,trick_cards,total_plays,attempts,successes,failures,retry_count,success_rate,avg_attempts_per_play,avg_retries_per_play,total_cost,total_revenue,net_profit,profit_per_play
0,1,2,5.0,2,1,0.8,0,23694,23694,20265,3429,3429,0.855280,1.0,0.144720,40530,1694,-38836,-1.639065
1,2,3,7.0,2,1,0.9,0,22942,22942,17561,5381,5381,0.765452,1.0,0.234548,35122,2677,-32445,-1.414218
2,3,6,9.0,2,1,1.0,0,30376,30376,4939,25437,25437,0.162595,1.0,0.837405,9878,12815,2937,0.096688
3,4,8,13.0,2,1,1.1,0,8909,8909,1176,7733,7733,0.132001,1.0,0.867999,2352,3970,1618,0.181614
4,5,8,15.0,3,1,1.2,2,2168,2168,133,2035,2035,0.061347,1.0,0.938653,399,992,593,0.273524
5,6,10,21.0,3,1,1.3,0,222,222,71,151,151,0.319820,1.0,0.680180,213,89,-124,-0.558559
6,7,10,25.0,6,1,1.4,2,120,120,29,91,91,0.241667,1.0,0.758333,174,49,-125,-1.041667


In [114]:
level_df.total_revenue.sum(), level_df.total_cost.sum()

(22555, 88973)

In [108]:
level_df.total_revenue.sum(), level_df.total_cost.sum()

(5475, 76997)

In [75]:
user_df.total_failures.sum(), user_df.total_retries.sum(), user_df.total_abandons.sum()
# level_df.failures.sum(), level_df.retry_count.sum()

(44257, 22286, 21971)

In [54]:
user_df.query('memory_size == 10').final_level.value_counts(normalize=True).sort_index()

final_level
1    0.079012
2    0.093827
3    0.558025
4    0.182716
5    0.074074
6    0.004938
7    0.007407
Name: proportion, dtype: float64

In [53]:
user_df.query('memory_size == 1').final_level.value_counts(normalize=True).sort_index()


final_level
1    0.188341
2    0.334081
3    0.466368
4    0.011211
Name: proportion, dtype: float64

In [28]:

user_df.query('memory_size == 5').final_level.value_counts(normalize=True).sort_index()

final_level
1    0.062929
2    0.116579
3    0.593788
4    0.180315
5    0.044776
6    0.001210
7    0.000403
Name: proportion, dtype: float64

In [34]:
user_df.query('memory_size == 10').final_level.value_counts().sort_index()

final_level
1     13
2     14
3    102
4     33
5     13
7      2
Name: count, dtype: int64

In [116]:
def find_break_even_point(simulator, prob_df, num_users=10000, max_iterations=20):
    """
    BEP를 찾기 위해 판당 보상을 조절하면서 시뮬레이션 실행
    
    Args:
        simulator: GameSimulator 인스턴스
        prob_df: 확률 분포 DataFrame
        num_users: 시뮬레이션 유저 수
        max_iterations: 최대 반복 횟수
    
    Returns:
        Dict: BEP 결과
    """
    print("=== BEP(Break-Even Point) 탐색 시작 ===")
    print("="*60)
    
    # 현재 보상 설정 저장
    original_rewards = {}
    for level, game_level in simulator.game_levels.items():
        original_rewards[level] = game_level.success_reward
    
    results = []
    
    # 낮은 판부터 보상 1씩 줄여가며 탐색
    for iteration in range(max_iterations):
        print(f"\n🔍 반복 {iteration + 1}: 보상 조정 중...")
        
        # 보상 조정 (낮은 판부터 1씩 감소)
        for level in range(1, len(simulator.game_levels) + 1):
            current_reward = original_rewards[level]
            adjusted_reward = max(1, current_reward - iteration)  # 최소 1원
            simulator.game_levels[level].success_reward = adjusted_reward
        
        # 현재 보상 설정 출력
        print("현재 보상 설정:")
        for level in range(1, len(simulator.game_levels) + 1):
            reward = simulator.game_levels[level].success_reward
            print(f"  Level {level}: {reward}원", end="  ")
        print()
        
        # 시뮬레이션 실행
        try:
            user_df, level_df = simulator.run_simulation(num_users=num_users)
            
            # 수익성 계산
            total_revenue = user_df['total_revenue'].sum()
            total_cost = user_df['total_cost'].sum()
            net_profit = total_revenue - total_cost
            profit_margin = (net_profit / total_revenue * 100) if total_revenue > 0 else 0
            
            # 결과 저장
            result = {
                'iteration': iteration + 1,
                'rewards': {level: simulator.game_levels[level].success_reward 
                           for level in range(1, len(simulator.game_levels) + 1)},
                'total_revenue': total_revenue,
                'total_cost': total_cost,
                'net_profit': net_profit,
                'profit_margin': profit_margin,
                'is_break_even': net_profit >= 0,
                'user_df': user_df,
                'level_df': level_df
            }
            results.append(result)
            
            print(f"💰 수익성: {total_revenue:,.0f}원 (수익) - {total_cost:,.0f}원 (비용) = {net_profit:,.0f}원")
            print(f"�� 마진: {profit_margin:.1f}%")
            
            # BEP 달성 확인
            if net_profit >= 0:
                print(f"🎯 BEP 달성! 반복 {iteration + 1}에서 손익분기점 도달")
                break
            else:
                print(f"❌ 아직 손실 상태 (손실: {abs(net_profit):,.0f}원)")
                
        except Exception as e:
            print(f"❌ 시뮬레이션 오류: {e}")
            break
    
    # 최종 결과 분석
    print("\n" + "="*60)
    print("�� BEP 탐색 결과 요약")
    print("="*60)
    
    if results:
        # BEP 달성한 결과 찾기
        bep_results = [r for r in results if r['is_break_even']]
        
        if bep_results:
            best_result = bep_results[0]  # 첫 번째 BEP 달성 결과
            print(f"✅ BEP 달성: 반복 {best_result['iteration']}")
            print(f"💰 최종 수익: {best_result['net_profit']:,.0f}원")
            print(f"�� 마진: {best_result['profit_margin']:.1f}%")
            print("\n🎯 최적 보상 설정:")
            for level, reward in best_result['rewards'].items():
                original = original_rewards[level]
                change = reward - original
                print(f"  Level {level}: {reward}원 (원래 {original}원, {change:+d}원)")
        else:
            print("❌ BEP 달성 실패: 최대 반복 횟수 내에서 손익분기점 도달 불가")
            last_result = results[-1]
            print(f"💰 최종 손실: {abs(last_result['net_profit']):,.0f}원")
            print(f"�� 마진: {last_result['profit_margin']:.1f}%")
    
    # 보상 원복
    for level, game_level in simulator.game_levels.items():
        game_level.success_reward = original_rewards[level]
    
    return results

def analyze_reward_sensitivity(results):
    """보상 민감도 분석"""
    print("\n" + "="*60)
    print("📊 보상 민감도 분석")
    print("="*60)
    
    if not results:
        print("분석할 결과가 없습니다.")
        return
    
    # 반복별 수익성 변화
    print("\n반복별 수익성 변화:")
    print("반복 | 수익(원) | 비용(원) | 순이익(원) | 마진(%)")
    print("-" * 50)
    
    for result in results:
        print(f"{result['iteration']:4d} | {result['total_revenue']:8,.0f} | {result['total_cost']:8,.0f} | "
              f"{result['net_profit']:9,.0f} | {result['profit_margin']:6.1f}")
    
    # 보상별 수익성 분석
    print("\n레벨별 보상 변화:")
    levels = list(range(1, len(results[0]['rewards']) + 1))
    print("반복 | " + " | ".join([f"L{i}" for i in levels]))
    print("-" * (6 + 4 * len(levels)))
    
    for result in results:
        rewards_str = " | ".join([f"{result['rewards'][i]:2d}" for i in levels])
        print(f"{result['iteration']:4d} | {rewards_str}")

# 사용 예시
def run_bep_analysis():
    """BEP 분석 실행"""
    # 1. 기본 시뮬레이터 생성
    simulator = GameSimulator(prob_df)
    
    # 2. BEP 탐색 실행
    results = find_break_even_point(simulator, prob_df, num_users=5000, max_iterations=15)
    
    # 3. 민감도 분석
    analyze_reward_sensitivity(results)
    
    return results

# 실행
bep_results = run_bep_analysis()

=== BEP(Break-Even Point) 탐색 시작 ===

🔍 반복 1: 보상 조정 중...
현재 보상 설정:
  Level 1: 2원    Level 2: 2원    Level 3: 2원    Level 4: 2원    Level 5: 3원    Level 6: 3원    Level 7: 6원  
=== 5000명 유저 게임 시뮬레이션 시작 ===
진행률: 1000/5000 (20.0%)
진행률: 2000/5000 (40.0%)
진행률: 3000/5000 (60.0%)
진행률: 4000/5000 (80.0%)
진행률: 5000/5000 (100.0%)
💰 수익성: 5,046원 (수익) - 19,986원 (비용) = -14,940원
�� 마진: -296.1%
❌ 아직 손실 상태 (손실: 14,940원)

🔍 반복 2: 보상 조정 중...
현재 보상 설정:
  Level 1: 1원    Level 2: 1원    Level 3: 1원    Level 4: 1원    Level 5: 2원    Level 6: 2원    Level 7: 5원  
=== 5000명 유저 게임 시뮬레이션 시작 ===
진행률: 1000/5000 (20.0%)
진행률: 2000/5000 (40.0%)
진행률: 3000/5000 (60.0%)
진행률: 4000/5000 (80.0%)
진행률: 5000/5000 (100.0%)
💰 수익성: 5,100원 (수익) - 10,218원 (비용) = -5,118원
�� 마진: -100.4%
❌ 아직 손실 상태 (손실: 5,118원)

🔍 반복 3: 보상 조정 중...
현재 보상 설정:
  Level 1: 1원    Level 2: 1원    Level 3: 1원    Level 4: 1원    Level 5: 1원    Level 6: 1원    Level 7: 4원  
=== 5000명 유저 게임 시뮬레이션 시작 ===
진행률: 1000/5000 (20.0%)
진행률: 2000/5000 (40.0%)
진행률: 3000/5000 (60.0%)


In [117]:
# bep_results[1].keys()

import pandas as pd

# bep_results의 반복별 수익성 정보를 DataFrame으로 저장
pd.DataFrame([{
    '반복': result['iteration'],
    '수익(원)': result['total_revenue'],
    '비용(원)': result['total_cost'],
    '순이익(원)': result['net_profit'],
    '마진(%)': result['profit_margin']
} for result in bep_results])

,반복,수익(원),비용(원),순이익(원),마진(%)
0,1,5046,19986,-14940,-296.076100
1,2,5100,10218,-5118,-100.352941
2,3,5241,10173,-4932,-94.104179
3,4,5058,10112,-5054,-99.920917
4,5,4914,10055,-5141,-104.619455
5,6,5137,9985,-4848,-94.374148
6,7,4711,9932,-5221,-110.825727
7,8,5135,10082,-4947,-96.338851
8,9,4947,10118,-5171,-104.527997
9,10,5114,10125,-5011,-97.985921


In [118]:
key = list(range(1, len(bep_results[0]['rewards']) + 1))
pd.DataFrame([bep_results[i-1]['rewards'] for i in key])

,1,2,3,4,5,6,7
0,2,2,2,2,3,3,6
1,1,1,1,1,2,2,5
2,1,1,1,1,1,1,4
3,1,1,1,1,1,1,3
4,1,1,1,1,1,1,2
5,1,1,1,1,1,1,1
6,1,1,1,1,1,1,1
